# Auswertung FFC-App
Das Notebook dient zur Auswertung der Daten, die durch die FFC-App gesammelt wurden. Die Daten wurden in einer CSV-Datei gespeichert und werden hier eingelesen und ausgewertet. Das Notebook wird für den Abschluss der Challenge genutzt, um die Ergebnisse zu präsentieren und über Verbesserungsmöglichkeiten zu für nächstes Jahr zu diskutieren.

In [413]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [414]:
types_of_fish_df = pd.read_csv('data/types_of_fish.csv')
types_of_fish_df.head()

,id,type,avg_length,upper_bound,lower_bound,is_rare
0,0,Aland,25,38,19,1
1,1,Schleie,25,38,19,0
2,2,Rotauge,20,30,15,0
3,3,Rotfeder,20,30,15,0
4,4,Ukelei,12,18,9,1


In [415]:
users_df = pd.read_csv('data/users.csv')
users_df.head()

,id,username,hashed_password,salt,name,email
0,1,alice,\xe0460026879f66523c5deed750bc059d70b2f896d902...,\xf8965e425407e79853816ba6bb05ebb0,NaN,NaN
1,2,Marvin,\x373414315838ffad43983814655cf59c7e17b33f19b4...,\xbf53d48f795c4eacfbf2a1b0aafa8aab,Marvin,marvin.bathke@gmx.de
2,3,dennisde,\x0daadf204f89c89416198643de321e7f32b8f210932f...,\x74e3474d31291fb5b38442b9c0b9a618,Dennis,dennisdeutsch@gmx.net
3,4,AlxDeeee,\x8580c4ac6b00786b88d477654c3bde6c25535ccef643...,\x3dfffae11dba983f930f5d656352c488,Alex,alex.deutsch@gmx.net
4,5,valeride,\x586ff0fffef7127d15338628d6cd8c8319acfb9ab2bf...,\xd25c24858f4eb3eb4226e1266bbb1708,Valeri,valerideutsch969@web.de


In [416]:
scoreboard_df = pd.read_csv('data/scoreboard.csv')
scoreboard_df.head()

,id,owner_id,fish_type_id,length,points,date
0,1,3,14,18,18.0,2024-03-02T11:57:58.817Z
1,2,3,14,18,18.0,2024-03-02T11:58:03.529Z
2,3,3,14,14,7.0,2024-03-02T12:42:16.352Z
3,4,3,14,12,6.0,2024-03-02T12:42:20.642Z
4,5,3,14,12,6.0,2024-03-02T12:42:26.218Z


In [417]:
merged_df = pd.merge(scoreboard_df, users_df, left_on='owner_id', right_on='id')
merged_df = pd.merge(merged_df, types_of_fish_df, left_on='fish_type_id', right_on='id')
#drop all columns execpt id_x, name, type, length, points and data
merged_df = merged_df[[  'length', 'points', 'date', 'name',  'type',
       'avg_length', 'upper_bound', 'lower_bound', 'is_rare']]
merged_df.head()

,length,points,date,name,type,avg_length,upper_bound,lower_bound,is_rare
0,18,18.0,2024-03-02T11:57:58.817Z,Dennis,Zwergwels,15,23,11,0
1,18,18.0,2024-03-02T11:58:03.529Z,Dennis,Zwergwels,15,23,11,0
2,14,7.0,2024-03-02T12:42:16.352Z,Dennis,Zwergwels,15,23,11,0
3,12,6.0,2024-03-02T12:42:20.642Z,Dennis,Zwergwels,15,23,11,0
4,12,6.0,2024-03-02T12:42:26.218Z,Dennis,Zwergwels,15,23,11,0


In [418]:
#format date column to DD.MM.YYYY
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.strftime('%d.%m.%Y')
df = merged_df
df.head()

,length,points,date,name,type,avg_length,upper_bound,lower_bound,is_rare
0,18,18.0,02.03.2024,Dennis,Zwergwels,15,23,11,0
1,18,18.0,02.03.2024,Dennis,Zwergwels,15,23,11,0
2,14,7.0,02.03.2024,Dennis,Zwergwels,15,23,11,0
3,12,6.0,02.03.2024,Dennis,Zwergwels,15,23,11,0
4,12,6.0,02.03.2024,Dennis,Zwergwels,15,23,11,0


In [419]:
user_colors = {
    'Alex':   '#1f77b4',  # Blau
    'Dennis': '#ff7f0e',  # Orange
    'Marvin': '#2ca02c',  # Grün
    'Valeri': '#d62728',  # Rot
}

# Grafiken

##  Wer hat die meisten Punkte erzielt?

In [420]:

# 3. Gruppiere nach 'name' und summiere 'points'
agg_points_df = df.groupby('name', as_index=False)['points'].sum()

# 4. Sortiere die aggregierten Daten in absteigender Reihenfolge
agg_points_df = agg_points_df.sort_values(by='points', ascending=False)

# 5. Erstelle das Balkendiagramm mit Plotly Express
fig = px.bar(
    agg_points_df,
    x='name',
    y='points',
    color='name',
    color_discrete_map=user_colors,  # Verwende das definierte Farbzuordnung
    text='points',                   # Füge Punkte als Beschriftung hinzu
    title='Gesamtpunkte pro Teilnehmer',
    labels={
        'name': 'Teilnehmer',
        'points': 'Gesamtpunkte'
    }
)

# 6. Passe das Layout an
fig.update_layout(
    xaxis={'categoryorder': 'total descending'},
    title={
        'text': "Gesamtpunkte pro Teilnehmer",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Teilnehmer',
    yaxis_title='Gesamtpunkte',
    legend_title_text='Teilnehmer',
    template='plotly_white'  # Weißes Hintergrundthema für bessere Ästhetik
)

# 7. Füge Datenbeschriftungen oberhalb der Balken hinzu
fig.update_traces(texttemplate='%{text}', textposition='outside')

# 8. Zeige das Diagramm an
fig.show()

## Wer hat mit welchen Fischen wie viele Punkte gemacht?

In [421]:
agg_df = df.groupby(['name', 'type'], as_index=False)['points'].sum()

agg_df = agg_df.sort_values(by='points', ascending=False)

fig = px.scatter(
    agg_df,
    x='name',
    y='type',
    size='points',
    color='name',
    color_discrete_map=user_colors,
    title='Anzahl der gefangenen Fische pro Benutzer und Fischart',
    labels={
        'name': 'Benutzer',
        'type': 'Fischart',
        'points': 'Anzahl der Punkte'
    },
    hover_name='name',
    hover_data=['type', 'points'],
    size_max=40  # Maximale Blasengröße / anpasssen falls Bildschirm anders
)
 
fig.update_layout(
    xaxis_title='Teilnehmer',
    yaxis_title='Fischart',
    legend_title='Teilnehmer',
    title_font_size=20,
    width=1000,   
    height=800,  
    margin=dict(l=50, r=150, t=80, b=50) 
)

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Wie oft wurden Fische gefangen?

In [422]:

# Gruppiere nach 'name' und zähle die einzigartigen 'date'-Einträge
agg_df = df.groupby('name', as_index=False)['date'].nunique()

agg_df.rename(columns={'date': 'einzigartige_tage'}, inplace=True)

agg_df = agg_df.sort_values(by='einzigartige_tage', ascending=False)

fig = px.bar(
    agg_df,
    x='name',
    y='einzigartige_tage',
    color='name',
    color_discrete_map=user_colors,  
    text='einzigartige_tage',         
    title='Anzahl der Fangtage pro Benutzer',
    labels={
        'name': 'Benutzer',
        'einzigartige_tage': 'Anzahl der Fangtage'
    }
)

fig.update_layout(
    xaxis={'categoryorder': 'total descending'},
    title={
        'text': "Anzahl der Fangtage pro Benutzer",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Benutzer',
    yaxis_title='Anzahl der Fangtage',
    legend_title_text='Benutzer',
    template='plotly_white'  
)

# Füge Datenbeschriftungen oberhalb der Balken hinzu
fig.update_traces(texttemplate='%{text}', textposition='outside')

# Zeige das Diagramm an
fig.show()

## Wie viele Punkte wurden pro Tag pro Teilnehmer erzielt?

In [423]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

daily_points = df.groupby(['name', 'date'], as_index=False)['points'].sum()

avg_points_per_day_df = daily_points.groupby('name', as_index=False)['points'].mean()

avg_points_per_day_df.rename(columns={'points': 'avg_points_per_day'}, inplace=True)

avg_points_per_day_df = avg_points_per_day_df.sort_values(by='avg_points_per_day', ascending=False)

fig = px.bar(
    avg_points_per_day_df,
    x='name',
    y='avg_points_per_day',
    color='name',
    color_discrete_map=user_colors,
    text='avg_points_per_day',      
    title='Durchschnittliche Punkte pro Tag und Benutzer',
    labels={
        'name': 'Benutzer',
        'avg_points_per_day': 'Durchschnittliche Punkte pro Tag'
    }
)

fig.update_layout(
    xaxis_title='Benutzer',
    yaxis_title='Durchschnittliche Punkte pro Tag',
    legend_title='Benutzer',
    template='plotly_white',  
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50)  
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Größter Fisch pro Art + geclustert nach Teilnehmer

In [425]:
max_length_df = df.loc[df.groupby('type')['length'].idxmax()].reset_index(drop=True)

max_length_df = max_length_df.sort_values(by='length', ascending=False)

fig = px.bar(
    max_length_df,
    x='type',
    y='length',
    color='name',
    color_discrete_map=user_colors,  
    text='length',                  
    title='Größter Fang pro Fischart',
    labels={
        'type': 'Fischart',
        'length': 'Länge (cm)',
        'name': 'Benutzer'
    }
)

fig.update_layout(
    xaxis_title='Fischart',
    yaxis_title='Länge (cm)',
    legend_title='Benutzer',
    template='plotly_white', 
    title={
        'text': "Größter Fang pro Fischart",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50)  
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Wie viele Fische wurden pro Art gefangen?

In [426]:

type_counts_df = df['type'].value_counts().reset_index()
type_counts_df.columns = ['type', 'anzahl_fänge']


fig = px.bar(
    type_counts_df,
    x='type',
    y='anzahl_fänge',
    color='type',  
    title='Anzahl der Fänge pro Fischart',
    labels={
        'type': 'Fischart',
        'anzahl_fänge': 'Anzahl der Fänge'
    },
    text='anzahl_fänge'  
)

fig.update_layout(
    xaxis_title='Fischart',
    yaxis_title='Anzahl der Fänge',
    legend_title='Fischart',
    template='plotly_white', 
    title={
        'text': "Anzahl der Fänge pro Fischart",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50) 
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Wie viele Fische wurden pro User gefangen?

In [427]:

agg_catches_df = df.groupby('name', as_index=False).size()
agg_catches_df.columns = ['name', 'anzahl_fänge']

agg_catches_df = agg_catches_df.sort_values(by='anzahl_fänge', ascending=False)

fig = px.bar(
    agg_catches_df,
    x='name',
    y='anzahl_fänge',
    color='name',
    color_discrete_map=user_colors, 
    text='anzahl_fänge',            
    title='Anzahl der Fänge pro Benutzer',
    labels={
        'name': 'Benutzer',
        'anzahl_fänge': 'Anzahl der Fänge'
    }
)

fig.update_layout(
    xaxis={'categoryorder': 'total descending'},  
    title={
        'text': "Anzahl der Fänge pro Benutzer",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Benutzer',
    yaxis_title='Anzahl der Fänge',
    legend_title='Benutzer',
    template='plotly_white',  
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50)  
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Wie viel Meter Fisch wurde pro Teilnehmer gefangen?

In [428]:

agg_length_df = df.groupby('name', as_index=False)['length'].sum()

agg_length_df = agg_length_df.sort_values(by='length', ascending=False)

fig = px.bar(
    agg_length_df,
    x='name',
    y='length',
    color='name',
    color_discrete_map=user_colors,  
    text='length',                   
    title='Gesamtlänge der Fänge pro Benutzer',
    labels={
        'name': 'Benutzer',
        'length': 'Gesamtlänge (cm)'
    }
)

fig.update_layout(
    xaxis={'categoryorder': 'total descending'},  
    title={
        'text': "Gesamtlänge der Fänge pro Benutzer",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Benutzer',
    yaxis_title='Gesamtlänge (cm)',
    legend_title='Benutzer',
    template='plotly_white', 
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50)  
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Wer ist der Gewinner / Verlierer durch Bonuspunkte?

In [429]:

agg_df = df.groupby('name', as_index=False).agg({
    'length': 'sum',
    'points': 'sum'
})

agg_df['differenz'] = agg_df['length'] - agg_df['points']

agg_df = agg_df.sort_values(by='differenz', ascending=False)


fig = px.bar(
    agg_df,
    x='name',
    y='differenz',
    color='name',
    color_discrete_map=user_colors,  
    text='differenz',                
    title='Differenz zwischen Gesamtlänge und Gesamtpunkten pro Benutzer',
    labels={
        'name': 'Benutzer',
        'differenz': 'Differenz (cm - Punkte)'
    }
)

fig.update_layout(
    xaxis={'categoryorder': 'total descending'},  
    title={
        'text': "Differenz zwischen Gesamtlänge und Gesamtpunkten pro Benutzer",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Benutzer',
    yaxis_title='Differenz (cm - Punkte)',
    legend_title='Benutzer',
    template='plotly_white', 
    width=800,
    height=600,
    margin=dict(l=50, r=200, t=100, b=50)  
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))

fig.show()

## Welche Fische haben die meisten Bonuspunkte / Abzüge erzielt?
Die Bewertung hier ist ein bisschen Kompliziert, da es per se keine Abzüge gab (Punkte konnten nicht ins negative gehe), aber die Differenz zwischen der Länge und der Punkte bezeichne ich hier als Abzug, da sie ohne da Punktesystem gezählt hätten

In [430]:
df['difference_length_to_points'] = df['length'] - df['points']
df['Bonuspunkte'] = df['points'] - df['length']

avg_diff_df = df.groupby('type', as_index=False)['Bonuspunkte'].mean().sort_values(by='Bonuspunkte', ascending=False)
avg_diff_df = avg_diff_df[avg_diff_df['Bonuspunkte'] != 0]
avg_diff_df['Bonuspunkte'] = avg_diff_df['Bonuspunkte'].round(2)

fig = px.bar(
    avg_diff_df,
    x='type',
    y='Bonuspunkte',
    color='type',
    text='Bonuspunkte',   
    title='Durchschnittliche Bonuspunkte pro Fischart',
    labels={
        'type': 'Fischart'
    }
    
)
fig.show()